In [1]:
from __future__ import division
import os
import sys
from cStringIO import StringIO

import numpy as np
import pandas as pd

from Bio import SeqIO

from subprocess import Popen,PIPE

from IPython.display import display
from ipywidgets import *

In [2]:
refgene = pd.read_csv('/DATA/raw/refseq/remap/top40_caps.txt', sep='\t').sort_values(by='name')

In [3]:
refgene = refgene[~refgene['chrom'].str.contains('_')]

In [4]:
refgene.head()

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames
0,1746,NM_001009931,chr1,-,152184551,152196672,152185551,152195729,3,"152184551,152195591,152196618,","152193966,152195754,152196672,",0,HRNR,cmpl,cmpl,"0,0,-1,"
20,1353,NM_001040105,chr7,+,100663363,100702140,100663416,100701325,13,"100663363,100674400,100674881,100691264,100692...","100663498,100674502,100687100,100691396,100692...",0,MUC17,cmpl,cmpl,"0,1,1,1,1,0,2,1,1,2,2,1,0,"
34,619,NM_001080400,chr19,-,4502191,4517716,4504470,4517716,6,"4502191,4504872,4508779,4510457,4516628,4517565,","4504797,4504959,4508967,4513716,4516690,4517716,",0,PLIN4,cmpl,cmpl,"0,0,1,0,1,0,"
35,1005,NM_001081637,chr19,+,55141967,55148981,55142524,55148329,15,"55141967,55142476,55142721,55142950,55143385,5...","55142084,55142558,55142757,55143238,55143688,5...",0,LILRB1,cmpl,cmpl,"-1,0,1,1,1,1,1,1,1,1,1,2,1,0,0,"
1,290,NM_001098623,chr1,+,228395830,228566575,228399484,228566496,105,"228395830,228399466,228401141,228401874,228402...","228395886,228400472,228401411,228402135,228402...",0,OBSCN,cmpl,cmpl,"-1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1..."


In [5]:
 def tabix_query(chrom, start, end, get_inds=False):
        filename = '/DATA/raw/GreatApes/vcfs/Pan_troglodytes.hg19.vcf.gz'
        query = 'chr{}:{}-{}'.format(chrom, start, end)
        if get_inds:
            process = Popen(['tabix', '-h', '-f', filename, query], stdout=PIPE)
            while True:
                line = process.stdout.readline().strip().split()
                if line[0][:2] != "##":
                    indIds = line[9:]
                    break
            yield indIds
        else:
            process = Popen(['tabix', '-f', filename, query], stdout=PIPE)
        for line in process.stdout:
            yield line.strip().split()


def chimp_genotypes(chrom,pos):
    if str(chrom)[:3] == "chr":
        chrom = chrom[3:]
    try:
        r = tabix_query(chrom,pos,pos)
        while True:
            res = r.next()
            if res[1] == str(pos):
                break
    except StopIteration:
        return None

    return res

In [6]:
def get_exons(row):
    es = map(int,row['exonStarts'].split(',')[:-1])
    ee = map(int,row['exonEnds'].split(',')[:-1])
    
    exons = []
    for start,end in zip(es,ee):
        if (row['cdsStart'] > start) and (row['cdsStart'] < end):
            start = row['cdsStart']
            exons.append((start,end))
        elif (row['cdsEnd'] < end) and (row['cdsEnd'] > start):
            end = row['cdsEnd']
            exons.append((start,end))
        elif (start >= row['cdsStart']) and (end <= row['cdsEnd']):
            exons.append((start,end))        
    return exons

In [7]:
def fill_sequences(sequences,base):
    for s in sequences:
        sequences[s].write(base)
    
    return sequences

In [8]:
def chunk_seq(seq, n=80):
    for i in range(0, len(seq), n):
        yield seq[i:i+n]

In [9]:
COMPLEMENTS = {'A':'T',
              'T':'A',
              'C':'G',
              'G':'C'}

In [10]:
individuals = tabix_query('1',1,999999999,get_inds=True).next()

In [11]:
individuals

['Pan_troglodytes_ellioti-Akwaya_Jean',
 'Pan_troglodytes_ellioti-Banyo',
 'Pan_troglodytes_ellioti-Basho',
 'Pan_troglodytes_ellioti-Damian',
 'Pan_troglodytes_ellioti-Julie',
 'Pan_troglodytes_ellioti-Kopongo',
 'Pan_troglodytes_ellioti-Koto',
 'Pan_troglodytes_ellioti-Paquita',
 'Pan_troglodytes_ellioti-Taweh',
 'Pan_troglodytes_ellioti-Tobi',
 'Pan_troglodytes_schweinfurthii-100037_Vincent',
 'Pan_troglodytes_schweinfurthii-100040_Andromeda',
 'Pan_troglodytes_schweinfurthii-9729_Harriet',
 'Pan_troglodytes_schweinfurthii-A910_Bwambale',
 'Pan_troglodytes_schweinfurthii-A911_Kidongo',
 'Pan_troglodytes_schweinfurthii-A912_Nakuu',
 'Pan_troglodytes_troglodytes-A957_Vaillant',
 'Pan_troglodytes_troglodytes-A958_Doris',
 'Pan_troglodytes_troglodytes-A959_Julie',
 'Pan_troglodytes_troglodytes-A960_Clara',
 'Pan_troglodytes_verus-9668_Bosco',
 'Pan_troglodytes_verus-9730_Donald',
 'Pan_troglodytes_verus-A956_Jimmie',
 'Pan_troglodytes_verus-Clint',
 'Pan_troglodytes_verus-X00100_Koby']

In [12]:
out_dir = '/DATA/raw/1KG/aln/chimp/'

In [13]:
%%time
prog = IntProgress(width='800px')
prog.color = '#CD5C5C'
prog.value = 0
prog.max = 1

pb = IntProgress(width='800px')
pb.value = 0
pb.max = refgene.shape[0]
ph = HTML()
ph.value = '{} / {}'.format(pb.value, pb.max)

display(prog,pb,ph)

for ix,row in refgene.iterrows():
    sequences = {}
    for ind in individuals:
        i1 = '{}.1'.format(ind)
        i2 = '{}.2'.format(ind)
        sequences[i1] = StringIO()
        sequences[i2] = StringIO()
    
    refcore = row['name']
    chrom = row['chrom']
    strand = row['strand']
    
    if os.path.isfile(os.path.join(out_dir,'{}.fas'.format(refcore))):
        pb.value += 1
        ph.value = '{} / {}'.format(pb.value, pb.max)
        continue
    
    
    ## Get exons
    res = get_exons(row)
    positions = []
    for r in res:
        positions.extend(range(r[0]+1,r[1]+1))
    
    seq = list(SeqIO.parse(os.path.join('/DATA/raw/refseq/46way/nuc_all/','{}.fas'.format(refcore)),'fasta').next().seq)
    
    if strand == '-':
        ps = zip(range(1,len(positions)+1),positions[::-1],seq)
    else:
        ps = zip(range(1,len(positions)+1),positions,seq)
        
#     p[refcore] = ps    
#     print refcore,len(ps)
#     continue
    
    d = dict([(x[0],(x[1],x[2])) for x in ps])
    
    ## Make sequences
    prog.value = 0
    prog.max = len(positions)+1
    
    for nuc_pos in range(1,len(positions)+1):
        prog.value += 1
        
        try:
            chrom_pos,base = d[nuc_pos]
        except:
#             print row,nuc_pos
            print '.',
            continue
        base = base.upper()
        snp = chimp_genotypes(chrom,chrom_pos)
        
        ## Empty results
        if snp is None:
            sequences = fill_sequences(sequences,base)
            continue

            ## No indels
        if (len(snp[3]) != 1) or (len(snp[4]) != 1):
            sequences = fill_sequences(sequences,base)
            continue
        
        ## Has data
        if strand == '+':
            ref = snp[3]
            alt = snp[4]
        else:
            ref = COMPLEMENTS[snp[3]]
            alt = COMPLEMENTS[snp[4]]

        for ind,gt in zip(individuals,snp[9:]):
            g1,g2 = gt.split(':')[0].split('/')
            if g1 == '.':
                g1 = '0'
            if g2 == '.':
                g2 = '0'
            sequences['{}.1'.format(ind)].write(g1.replace('0',ref).replace('1',alt))
            sequences['{}.2'.format(ind)].write(g2.replace('0',ref).replace('1',alt))
    
    with open(os.path.join(out_dir,'{}.fas'.format(refcore)),'w') as fh:
        fh.write('>hg19\n{}\n'.format('\n'.join(chunk_seq(''.join(seq)))))
        
        for ind in individuals:
            i1 = '{}.1'.format(ind)
            i2 = '{}.2'.format(ind)

            fh.write('>{}\n{}\n'.format(i1,'\n'.join(chunk_seq(sequences[i1].getvalue()))))
            fh.write('>{}\n{}\n'.format(i2,'\n'.join(chunk_seq(sequences[i2].getvalue()))))
    
    pb.value += 1
    ph.value = '{} / {}'.format(pb.value, pb.max)


. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 